In [2]:
def add_code_to_file(file_path, code_block):
    try:
        # Open the file in append mode and add the code block
        with open(file_path, 'a') as file:
            file.write("\n")  # Ensure the block starts on a new line
            file.write(code_block)
        
        print(f"Code block added successfully to '{file_path}'.")

    except FileNotFoundError:
        print(f"The file {file_path} was not found.")
    except Exception as e:
        print(f"An error occurred: {e}")

# Add code block
code_block = """+PROG ASE urs:9 $ Linear Analysis
HEAD Calculation of forces and moments
PAGE UNII 0
CTRL OPT WARP VAL 0
LC ALL
END
+PROG WING urs:9.1 $ Graphical Output
HEAD Graphical Output
PAGE UNII 0
CTRL EMPT YES         $ create empty pages if results not available
CTRL WARN (800 802 1) $ no warnings if no values found
CTRL WARN (804 808 1) $ no warnings if no values found
CTRL WARN 873         $ no warning for 2D visibility
#define SCHR=0.2
SCHH H6 0.2
#define FILL=-
#define FILLI=-
#define FILLC=-
#define SCHRI=-
#define SCHRC=-
SIZ2 SPLI PICT
SIZE -URS SC 0 SPLI  2x1 MARG NO FORM STAN
VIEW EG3
LC 1 DESI 1
LOAD TYPE ALL
LC 2 DESI 2
LOAD TYPE ALL
LC 3 DESI 3
LOAD TYPE ALL
LC 1 DESI 1
NODE TYPE SV SCHH YES
LC 2 DESI 2
NODE TYPE SV SCHH YES
LC 3 DESI 3
NODE TYPE SV SCHH YES
LC 1 DESI 1
DEFO TYPE FULL FAC DEFA LC CURR; STRU NUME 0 0; DEFO NO
LC 2 DESI 2
DEFO TYPE FULL FAC DEFA LC CURR; STRU NUME 0 0; DEFO NO
LC 3 DESI 3
DEFO TYPE FULL FAC DEFA LC CURR; STRU NUME 0 0; DEFO NO
LC 1 DESI 1
BEAM TYPE MY
LC 2 DESI 2
BEAM TYPE MY
LC 3 DESI 3
BEAM TYPE MY
LC 1 DESI 1
BEAM TYPE MZ
LC 2 DESI 2
BEAM TYPE MZ
LC 3 DESI 3
BEAM TYPE MZ
LC 1 DESI 1
BEAM TYPE MT
LC 2 DESI 2
BEAM TYPE MT
LC 3 DESI 3
BEAM TYPE MT
LC 1 DESI 1
BEAM TYPE VZ
LC 2 DESI 2
BEAM TYPE VZ
LC 3 DESI 3
BEAM TYPE VZ
LC 1 DESI 1
BEAM TYPE VY
LC 2 DESI 2
BEAM TYPE VY
LC 3 DESI 3
BEAM TYPE VY
LC 1 DESI 1
BEAM TYPE  N
LC 2 DESI 2
BEAM TYPE  N
LC 3 DESI 3
BEAM TYPE  N
END"""

file_path = r"C:\Users\Côme Delecourt\Desktop\TESTS\Sophistik\barres_exp.dat"
add_code_to_file(file_path, code_block)

Code block added successfully to 'C:\Users\Côme Delecourt\Desktop\TESTS\Sophistik\barres_exp.dat'.


In [6]:
import re

def extract_value_after_string(file_path, string):
    try:
        # Read the file content
        with open(file_path, 'r') as file:
            lines = file.readlines()

        # Iterate through lines to find the matching string and extract the value
        for line in lines:
            if string in line:
                # Use regular expression to find the value immediately after the string
                match = re.search(rf'{re.escape(string)}\s+([-+]?\d*\.?\d+)', line)
                if match:
                    value = match.group(1)  # Extract the value found
                    print(f"Value after '{string}': {value}")
                    return value
                else:
                    print(f"No value found after '{string}'")
                    return None

    except FileNotFoundError:
        print(f"The file {file_path} was not found.")
    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage:
file_path = r"C:\Users\Côme Delecourt\Desktop\TESTS\Sophistik\barres_exp.dat"
string = 'POIN AUTO WIDE 0 TYPE PG P'

value = extract_value_after_string(file_path, string)

def modify_file(file_path, string, str_a, str_b):
    try:
        # Read the file content
        with open(file_path, 'r') as file:
            lines = file.readlines()

        # Modify the specific line
        for i, line in enumerate(lines):
            if string in line:
                # Replace the specified string
                lines[i] = line.replace(str_a, str_b)
                print(f"Modified line {i+1}: {lines[i].strip()}")

        # Write the modified content back to the file
        with open(file_path, 'w') as file:
            file.writelines(lines)
        
        print(f"File '{file_path}' has been successfully modified.")
    
    except FileNotFoundError:
        print(f"The file {file_path} was not found.")
    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage:
file_path = r"C:\Users\Côme Delecourt\Desktop\TESTS\Sophistik\barres_exp.dat"

# Modify part
string = 'POIN AUTO WIDE 0 TYPE PG P'
str_a = ' 57 '
str_b = ' 7 '
modify_file(file_path, string, str_a, str_b)

Value after 'POIN AUTO WIDE 0 TYPE PG P': 57
Modified line 114: POIN AUTO WIDE 0 TYPE PG P 7 1.7285878
File 'C:\Users\Côme Delecourt\Desktop\TESTS\Sophistik\barres_exp.dat' has been successfully modified.


In [1]:
from sofistik_daten import *
import os                     # for the environment variable necessary
import platform               # checks the python platform
from ctypes import *          # read the functions from the cdb

# This example has been tested with Python 3.7 (64-bit)
print("The path variable=", os.environ["Path"])

# Check the python platform (32bit or 64bit)
print("Python architecture=", platform.architecture())
sofPlatform = str(platform.architecture())


if sofPlatform.find("32Bit") < 0:
    # Set environment variable for the dll files
    print ("Hint: 64bit DLLs are used")

    # Set DLL dir path - new in PY 3.8 for ctypes
    # See: https://docs.python.org/3/whatsnew/3.8.html#ctypes
    os.add_dll_directory(r"C:\Program Files\SOFiSTiK\2024\SOFiSTiK 2024\interfaces\64bit")
    os.add_dll_directory(r"C:\Program Files\SOFiSTiK\2024\SOFiSTiK 2024")

    # Get the DLL functions
    myDLL = cdll.LoadLibrary("sof_cdb_w-2024.dll")
    py_sof_cdb_get = cdll.LoadLibrary("sof_cdb_w-2024.dll").sof_cdb_get
    py_sof_cdb_get.restype = c_int

    py_sof_cdb_kenq = cdll.LoadLibrary("sof_cdb_w-2024.dll").sof_cdb_kenq_ex
else:
    # Set environment variable for the DLL files
    print ("Hint: 32bit DLLs are used")

    # Set DLL dir path - new in PY 3.8 for ctypes
    # See: https://docs.python.org/3/whatsnew/3.8.html#ctypes
    os.add_dll_directory(r"C:\Program Files\SOFiSTiK\2024\SOFiSTiK 2024\interfaces\32bit")
    os.add_dll_directory(r"C:\Program Files\SOFiSTiK\2024\SOFiSTiK 2024")

    # Get the DLL functions
    myDLL = cdll.LoadLibrary("cdb_w31.dll")
    py_sof_cdb_get = cdll.LoadLibrary("cdb_w31.dll").sof_cdb_get
    py_sof_cdb_get.restype = c_int

    py_sof_cdb_kenq = cdll.LoadLibrary("cdb_w31.dll").sof_cdb_kenq_ex


# Connect to CDB
Index = c_int()
cdbIndex = 99

# Set the CDB Path
# e.g. fileName = "S:\\test\\read_nodes.cdb"
fileName = r"C:\Users\Côme Delecourt\Desktop\TESTS\Sophistik\barres.cdb"

# important: Unicode call!
Index.value = myDLL.sof_cdb_init(fileName.encode('utf-8'), cdbIndex)

cdbStat = c_int()  # get the CDB status
cdbStat.value = myDLL.sof_cdb_status(Index.value)

# Print the Status of the CDB
print("CDB Status:", cdbStat.value)


ie = c_int(0)
RecLen = c_int(sizeof(cnode))

nodes = []
while ie.value < 2:
   ie.value = py_sof_cdb_get(Index, 20, 0, byref(cnode), byref(RecLen), 1)
   nodes.append([
      cnode.m_nr,      # node-number
      cnode.m_xyz[0],  # x coordinates
      cnode.m_xyz[1],  # y coordinates
      ])
   
   # Always read the length of record before sof_cdb_get is called
   RecLen = c_int(sizeof(cnode))

print(nodes)
print(len(nodes))

myDLL.sof_cdb_close(0)
# Print again the status of the CDB, if status = 0 -> CDB Closed successfully
cdbStat.value = myDLL.sof_cdb_status(Index.value)
if cdbStat.value == 0:
   print("CDB closed successfully, status = 0")

The path variable= c:\Anaconda3;C:\Anaconda3;C:\Anaconda3\Library\mingw-w64\bin;C:\Anaconda3\Library\usr\bin;C:\Anaconda3\Library\bin;C:\Anaconda3\Scripts;C:\Anaconda3\bin;C:\Anaconda3\condabin;C:\windows\system32;C:\windows;C:\windows\System32\Wbem;C:\windows\System32\WindowsPowerShell\v1.0;C:\windows\System32\OpenSSH;C:\Program Files\dotnet;C:\Program Files\Git\cmd;C:\Users\Côme Delecourt\AppData\Local\Microsoft\WindowsApps;C:\Users\Côme Delecourt\AppData\Local\Programs\Microsoft VS Code\bin
Python architecture= ('64bit', 'WindowsPE')
Hint: 64bit DLLs are used
CDB Status: 3
[[1, 0.0, 0.0], [1001, 0.0, 8.0], [1002, 1.7283830642700195, 8.0], [1003, 0.0, 0.6153846383094788], [1004, 0.0, 1.2307692766189575], [1005, 0.0, 1.8461538553237915], [1006, 0.0, 2.461538553237915], [1007, 0.0, 3.076923131942749], [1008, 0.0, 3.692307710647583], [1009, 0.0, 4.307692527770996], [1010, 0.0, 4.92307710647583], [1011, 0.0, 5.538461685180664], [1012, 0.0, 6.153846263885498], [1013, 0.0, 6.76923084259033